# Monads Revisited

Let's go back to monads. In Haskell, a monad is a type class defined as
```haskell
class Functor m => Monad m where
    (<=<) :: (b -> m c) -> (a -> m b) -> (a -> m c)
    return :: a -> m a
```

In [1]:
using Pkg
Pkg.activate(".")
using MLStyle

include("aux_functors.jl");

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/CategoryTheory`


## Example - List Monad

In [2]:
struct Nil end
struct Cons{T}
  val::T
  next::Union{Cons{T},Nil} # n.b. abstract type!
end
List{T} = Union{Cons{T}, Nil}
fmap(f::Function, x::Nil) = Nil()
fmap(f::Function, x::Cons) = Cons(f(x.val),fmap(f, x.next))

fmap (generic function with 4 methods)

In [44]:
concatenate(x::Nil ,y::List) = y
concatenate(x::Cons,y::List) = Cons(x.val, concatenate(x.next, y))

μ(x::Nil) = Nil()
μ(x::Cons{<:List})  = concatenate(x.val, μ(x.next))
η(x,::Type{List}) = Cons(x,Nil())
bind(x::List, f::Function) = μ(fmap(f, x))


bind (generic function with 1 method)

In [45]:
concatplus1(x::Int)::List  = Cons(x+1,Cons(x, Nil()))
x = Cons(1,Nil())

μ(fmap(concatplus1,bind(x, concatplus1)))

Cons{Int64}(3, Cons{Int64}(2, Cons{Int64}(2, Cons{Int64}(1, Nil()))))

## Example - Maybe Monad 

Remeber that `Maybe` is an endofunctor. If we have a natural transformation 
$\eta$ and $\mu$ such that $(\text{Maybe}, \eta, \mu)$ is a monoid, then we have our monad.

In [63]:
η(x,::Type{Maybe})   = Just(x)
μ(x::Just{<:Just})   = Just(x._1._1)
μ(x::Just{<:Nothing})= Nothing()
μ(x::Nothing)        = Nothing()

μ(Just(η(10, Maybe)))
bind(mx::Maybe, f::Function) = μ(fmap(f, mx))
⊙(f::Function,g::Function) = x -> bind(g(x),f)

safesqrt(x::Real) = x < 0 ? Nothing() : Just(√x)
safesolve(x::Real) = x == 0 ? Nothing() : Just(10 / x)
@show (safesolve ⊙ safesqrt)(14)
@show (safesolve ⊙ safesqrt)(-10)
@show (safesolve ⊙ safesqrt)(0);

(safesolve ⊙ safesqrt)(14) = Just{Float64}(2.6726124191242437)
(safesolve ⊙ safesqrt)(-10) = Nothing()
(safesolve ⊙ safesqrt)(0) = Nothing()


## Example - State Monad

In [110]:
# mapm(f::Function, x::Vector) = x == [] ? η([], Maybe) : [f(x[1]), mapm(f,x[2:end])]
# conv(x::String) = occursin(x,"0123456789") ? Just(parse(Int,x)) : Nothing()

In [109]:
mapm(conv, ["1","2"])

3-element Vector{Any}:
 Just{Int64}(1)
 Any[Just{Int64}(2), Just{Vector{Any}}(Any[]), Maybe]
 Maybe (alias for Union{Nothing, Just{T}} where T)